# Using translation attention ecoder-decocer to transform the b5 dataset

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
import pandas as pd
import unicodedata
import string
import re
import random
import copy
from contra_qa.plots.functions  import simple_step_plot, plot_confusion_matrix
import  matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from nltk.translate.bleu_score import sentence_bleu


% matplotlib inline

### Preparing data

In [2]:
df2 = pd.read_csv("data/boolean5_train.csv")
df2_test = pd.read_csv("data/boolean5_test.csv")

df2["text"] = df2["sentence1"] + df2["sentence2"] 
df2_test["text"] = df2_test["sentence1"] + df2_test["sentence2"] 

all_sentences = list(df2.text.values) + list(df2_test.text.values)

df2train = df2

In [3]:
df2train.tail()

,sentence1,sentence2,and_A,and_B,label,text
9995,Kristin created a victorious and ashy work of art,Kristin didn't create a victorious work of art,Kristin created a victorious work of art,Kristin created an ashy work of art,1,Kristin created a victorious and ashy work of ...
9996,Doris created an ashy and easy work of art,Wendy didn't create an easy work of art,Doris created an ashy work of art,Doris created an easy work of art,0,Doris created an ashy and easy work of art Wen...
9997,Dawn created a shapely and plain work of art,Wilma didn't create a plain work of art,Dawn created a shapely work of art,Dawn created a plain work of art,0,Dawn created a shapely and plain work of art W...
9998,Gertrude created an ugly and eager work of art,Gertrude didn't create an ugly work of art,Gertrude created an ugly work of art,Gertrude created an eager work of art,1,Gertrude created an ugly and eager work of art...
9999,Elizabeth created a gifted and worried work of...,Elizabeth didn't create a gifted work of art,Elizabeth created a gifted work of art,Elizabeth created a worried work of art,1,Elizabeth created a gifted and worried work of...


In [4]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


example = "ddddda'''~~çãpoeéééééÈ'''#$$##@!@!@AAS@#12323fdf"
print("Before:", example)
print()
print("After:", normalizeString(example))

Before: ddddda'''~~çãpoeéééééÈ'''#$$##@!@!@AAS@#12323fdf

After: ddddda capoeeeeeee ! ! aas fdf


In [6]:
pairs_A = list(zip(list(df2train.sentence1.values), list(df2train.and_A.values)))
pairs_B = list(zip(list(df2train.sentence1.values), list(df2train.and_B.values)))
pairs_A = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_A]
pairs_B = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in pairs_B]



In [7]:
all_text_pairs = zip(all_sentences, all_sentences)
all_text_pairs = [(normalizeString(s1), normalizeString(s2)) for s1, s2 in all_text_pairs]

In [8]:
def readLangs(lang1, lang2, pairs, reverse=False):
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [tuple(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [9]:
f = lambda x: len(x.split(" "))

MAX_LENGTH = np.max(list(map(f, all_sentences)))

In [10]:
MAX_LENGTH = 20

def filterPair(p):
    cond1 = len(p[0].split(' ')) < MAX_LENGTH
    cond2 = len(p[1].split(' ')) < MAX_LENGTH 
    return cond1 and cond2

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


In [11]:
def prepareData(lang1, lang2, pairs, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, pairs, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [12]:
_, _, training_pairs_A = prepareData("eng_enc",
                                             "eng_dec",
                                             pairs_A)

print()


input_lang, _, _ = prepareData("eng_enc",
                               "eng_dec",
                               all_text_pairs)

output_lang = copy.deepcopy(input_lang)


Read 10000 sentence pairs
Trimmed to 10000 sentence pairs
Counting words...
Counted words:
eng_enc 773
eng_dec 772

Read 11000 sentence pairs
Trimmed to 11000 sentence pairs
Counting words...
Counted words:
eng_enc 776
eng_dec 776


In [13]:
_, _, training_pairs_B = prepareData("eng_enc",
                                     "eng_dec",
                                     pairs_B)

Read 10000 sentence pairs
Trimmed to 10000 sentence pairs
Counting words...
Counted words:
eng_enc 773
eng_dec 772


### sentences 2 tensors

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

In [15]:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [16]:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [17]:
def tensorsFromTriple(triple):
    input_tensor = tensorFromSentence(input_lang, triple[0])
    target_tensor = tensorFromSentence(output_lang, triple[1])
    label_tensor = torch.tensor(triple[2], dtype=torch.long).view((1))
    return (input_tensor, target_tensor, label_tensor)

### models

In [18]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [19]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
hidden_size = 256
eng_enc_v_size = input_lang.n_words
eng_dec_v_size = output_lang.n_words

In [21]:
input_lang.n_words

776

In [22]:
encoderA = EncoderRNN(eng_enc_v_size, hidden_size)
decoderA = AttnDecoderRNN(hidden_size, eng_dec_v_size)
encoderA.load_state_dict(torch.load("b5_encoder1_att.pkl"))
decoderA.load_state_dict(torch.load("b5_decoder1_att.pkl"))

In [23]:
encoderB = EncoderRNN(eng_enc_v_size, hidden_size)
decoderB = AttnDecoderRNN(hidden_size, eng_dec_v_size)
encoderB.load_state_dict(torch.load("b5_encoder2_att.pkl"))
decoderB.load_state_dict(torch.load("b5_decoder2_att.pkl"))

## translating

In [24]:
def translate(encoder,
              decoder,
              sentence,
              max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(
            max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            _, topone = decoder_output.data.topk(1)
            if topone.item() == EOS_token:
                break
            else:
                decoded_words.append(output_lang.index2word[topone.item()])

            decoder_input = topone.squeeze().detach()

        return " ".join(decoded_words)

In [25]:
def projectA(sent):
    neural_translation = translate(encoderA,
                                   decoderA,
                                   sent,
                                   max_length=MAX_LENGTH)
    return neural_translation


def projectB(sent):
    neural_translation = translate(encoderB,
                                   decoderB,
                                   sent,
                                   max_length=MAX_LENGTH)
    return neural_translation

## translation of a trained model: and A

In [26]:
for t in training_pairs_A[0:3]:
    print("input_sentence : " + t[0])
    neural_translation = projectA(t[0])
    print("neural translation : " + neural_translation)
    reference = t[1]
    print("reference translation : " + reference)
    reference = reference.split(" ")
    candidate = neural_translation.split(" ")
    score = sentence_bleu([reference], candidate)
    print("blue score = {:.2f}".format(score))
    print()

input_sentence : jeffery created a silly and vast work of art
neural translation : brenda created a blue work of art
reference translation : jeffery created a silly work of art
blue score = 0.52

input_sentence : hilda created a zealous and better work of art
neural translation : brenda created a pitiful work of art
reference translation : hilda created a zealous work of art
blue score = 0.52

input_sentence : cheryl created an ugly and obedient work of art
neural translation : brenda created an ugly work of art
reference translation : cheryl created an ugly work of art
blue score = 0.81



/usr/local/lib/python3.5/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


## translation of a trained model: and B

In [27]:
for t in training_pairs_B[0:3]:
    print("input_sentence : " + t[0])
    neural_translation = projectB(t[0])
    print("neural translation : " + neural_translation)
    reference = t[1]
    print("reference translation : " + reference)
    reference = reference.split(" ")
    candidate = neural_translation.split(" ")
    score = sentence_bleu([reference], candidate)
    print("blue score = {:.2f}".format(score))
    print()

input_sentence : jeffery created a silly and vast work of art
neural translation : marion created a vast work of art
reference translation : jeffery created a vast work of art
blue score = 0.81

input_sentence : hilda created a zealous and better work of art
neural translation : marion created a better work of art
reference translation : hilda created a better work of art
blue score = 0.81

input_sentence : cheryl created an ugly and obedient work of art
neural translation : jessie created an obedient work of art
reference translation : cheryl created an obedient work of art
blue score = 0.81



## generating new data for training

In [28]:
df2train.sentence1 = df2train.sentence1.map(normalizeString)

In [29]:
df2train["project A"] = df2train.sentence1.map(projectA)

In [30]:
df2.head()

,sentence1,sentence2,and_A,and_B,label,text,project A
0,jeffery created a silly and vast work of art,Jeffery didn't create a silly work of art,Jeffery created a silly work of art,Jeffery created a vast work of art,1,Jeffery created a silly and vast work of art J...,brenda created a blue work of art
1,hilda created a zealous and better work of art,Hilda didn't create a zealous work of art,Hilda created a zealous work of art,Hilda created a better work of art,1,Hilda created a zealous and better work of art...,brenda created a vast work of art
2,cheryl created an ugly and obedient work of art,Cheryl didn't create an ugly work of art,Cheryl created an ugly work of art,Cheryl created an obedient work of art,1,Cheryl created an ugly and obedient work of ar...,brenda created an ugly work of art
3,hector created a plump and bald work of art,Hector didn't create a plump work of art,Hector created a plump work of art,Hector created a bald work of art,1,Hector created a plump and bald work of art He...,brenda created a kind work of art
4,madeline created a lemon and important work of...,Madeline didn't create a careful work of art,Madeline created a lemon work of art,Madeline created an important work of art,0,Madeline created a lemon and important work of...,brenda created a gray work of art


In [31]:
df2train["project B"] = df2train.sentence1.map(projectB)

In [32]:
df2.head()

,sentence1,sentence2,and_A,and_B,label,text,project A,project B
0,jeffery created a silly and vast work of art,Jeffery didn't create a silly work of art,Jeffery created a silly work of art,Jeffery created a vast work of art,1,Jeffery created a silly and vast work of art J...,brenda created a blue work of art,marion created a vast work of art
1,hilda created a zealous and better work of art,Hilda didn't create a zealous work of art,Hilda created a zealous work of art,Hilda created a better work of art,1,Hilda created a zealous and better work of art...,brenda created a vast work of art,marion created a better work of art
2,cheryl created an ugly and obedient work of art,Cheryl didn't create an ugly work of art,Cheryl created an ugly work of art,Cheryl created an obedient work of art,1,Cheryl created an ugly and obedient work of ar...,brenda created an ugly work of art,jessie created an obedient work of art
3,hector created a plump and bald work of art,Hector didn't create a plump work of art,Hector created a plump work of art,Hector created a bald work of art,1,Hector created a plump and bald work of art He...,brenda created a kind work of art,marion created a bald work of art
4,madeline created a lemon and important work of...,Madeline didn't create a careful work of art,Madeline created a lemon work of art,Madeline created an important work of art,0,Madeline created a lemon and important work of...,brenda created a gray work of art,marion created an important work of art


In [33]:
df2train["sentence1_p"] = df2train["project A"] + " and " + df2train["project B"]

In [34]:
df2.head()

,sentence1,sentence2,and_A,and_B,label,text,project A,project B,sentence1_p
0,jeffery created a silly and vast work of art,Jeffery didn't create a silly work of art,Jeffery created a silly work of art,Jeffery created a vast work of art,1,Jeffery created a silly and vast work of art J...,brenda created a blue work of art,marion created a vast work of art,brenda created a blue work of art and marion c...
1,hilda created a zealous and better work of art,Hilda didn't create a zealous work of art,Hilda created a zealous work of art,Hilda created a better work of art,1,Hilda created a zealous and better work of art...,brenda created a vast work of art,marion created a better work of art,brenda created a vast work of art and marion c...
2,cheryl created an ugly and obedient work of art,Cheryl didn't create an ugly work of art,Cheryl created an ugly work of art,Cheryl created an obedient work of art,1,Cheryl created an ugly and obedient work of ar...,brenda created an ugly work of art,jessie created an obedient work of art,brenda created an ugly work of art and jessie ...
3,hector created a plump and bald work of art,Hector didn't create a plump work of art,Hector created a plump work of art,Hector created a bald work of art,1,Hector created a plump and bald work of art He...,brenda created a kind work of art,marion created a bald work of art,brenda created a kind work of art and marion c...
4,madeline created a lemon and important work of...,Madeline didn't create a careful work of art,Madeline created a lemon work of art,Madeline created an important work of art,0,Madeline created a lemon and important work of...,brenda created a gray work of art,marion created an important work of art,brenda created a gray work of art and marion c...


In [35]:
df_train_plus = df2train[["sentence1_p", "sentence2", "label"]]

df_train_plus.sentence2 = df_train_plus.sentence2.map(normalizeString)

/home/felsal/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [36]:
df_train_plus.rename(columns={"sentence1_p": "sentence1"}, inplace=True)

/home/felsal/.local/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [37]:
df_train_plus.head()

,sentence1,sentence2,label
0,brenda created a blue work of art and marion c...,jeffery didn t create a silly work of art,1
1,brenda created a vast work of art and marion c...,hilda didn t create a zealous work of art,1
2,brenda created an ugly work of art and jessie ...,cheryl didn t create an ugly work of art,1
3,brenda created a kind work of art and marion c...,hector didn t create a plump work of art,1
4,brenda created a gray work of art and marion c...,madeline didn t create a careful work of art,0


In [38]:
df_train_plus.to_csv("data/boolean5_plus_train.csv", index=False)

## generating new data for test

In [39]:
df2_test.sentence1 = df2_test.sentence1.map(normalizeString)

In [40]:
df2_test["project A"] = df2_test.sentence1.map(projectA)

In [41]:
df2_test.head()

,sentence1,sentence2,and_A,and_B,label,text,project A
0,jared created an ashy and witty work of art,Alexander didn't create an ashy work of art,Jared created an ashy work of art,Jared created a witty work of art,0,Jared created an ashy and witty work of art Al...,gregory created an ashy work of art
1,patricia created a calm and icy work of art,Patricia didn't create a calm work of art,Patricia created a calm work of art,Patricia created an icy work of art,1,Patricia created a calm and icy work of art Pa...,brenda created a calm work of art
2,elsie created a beautiful and gorgeous work of...,Juana didn't create a beautiful work of art,Elsie created a beautiful work of art,Elsie created a gorgeous work of art,0,Elsie created a beautiful and gorgeous work of...,brenda created a beautiful work of art
3,herbert created a mysterious and worried work ...,Herbert didn't create a mysterious work of art,Herbert created a mysterious work of art,Herbert created a worried work of art,1,Herbert created a mysterious and worried work ...,brenda created a mango work of art
4,janice created an easy and agreeable work of art,Janice didn't create an easy work of art,Janice created an easy work of art,Janice created an agreeable work of art,1,Janice created an easy and agreeable work of a...,brenda created an easy work of art


In [42]:
df2_test["project B"] = df2_test.sentence1.map(projectB)

In [43]:
df2_test.head()

,sentence1,sentence2,and_A,and_B,label,text,project A,project B
0,jared created an ashy and witty work of art,Alexander didn't create an ashy work of art,Jared created an ashy work of art,Jared created a witty work of art,0,Jared created an ashy and witty work of art Al...,gregory created an ashy work of art,marion created a witty work of art
1,patricia created a calm and icy work of art,Patricia didn't create a calm work of art,Patricia created a calm work of art,Patricia created an icy work of art,1,Patricia created a calm and icy work of art Pa...,brenda created a calm work of art,ted created an icy work of art
2,elsie created a beautiful and gorgeous work of...,Juana didn't create a beautiful work of art,Elsie created a beautiful work of art,Elsie created a gorgeous work of art,0,Elsie created a beautiful and gorgeous work of...,brenda created a beautiful work of art,jessie created a gorgeous work of art
3,herbert created a mysterious and worried work ...,Herbert didn't create a mysterious work of art,Herbert created a mysterious work of art,Herbert created a worried work of art,1,Herbert created a mysterious and worried work ...,brenda created a mango work of art,jessie created a worried work of art
4,janice created an easy and agreeable work of art,Janice didn't create an easy work of art,Janice created an easy work of art,Janice created an agreeable work of art,1,Janice created an easy and agreeable work of a...,brenda created an easy work of art,jessie created an agreeable work of art


In [44]:
df2_test["sentence1_p"] = df2_test["project A"] + " and " + df2_test["project B"]

In [45]:
df2_test.head()

,sentence1,sentence2,and_A,and_B,label,text,project A,project B,sentence1_p
0,jared created an ashy and witty work of art,Alexander didn't create an ashy work of art,Jared created an ashy work of art,Jared created a witty work of art,0,Jared created an ashy and witty work of art Al...,gregory created an ashy work of art,marion created a witty work of art,gregory created an ashy work of art and marion...
1,patricia created a calm and icy work of art,Patricia didn't create a calm work of art,Patricia created a calm work of art,Patricia created an icy work of art,1,Patricia created a calm and icy work of art Pa...,brenda created a calm work of art,ted created an icy work of art,brenda created a calm work of art and ted crea...
2,elsie created a beautiful and gorgeous work of...,Juana didn't create a beautiful work of art,Elsie created a beautiful work of art,Elsie created a gorgeous work of art,0,Elsie created a beautiful and gorgeous work of...,brenda created a beautiful work of art,jessie created a gorgeous work of art,brenda created a beautiful work of art and jes...
3,herbert created a mysterious and worried work ...,Herbert didn't create a mysterious work of art,Herbert created a mysterious work of art,Herbert created a worried work of art,1,Herbert created a mysterious and worried work ...,brenda created a mango work of art,jessie created a worried work of art,brenda created a mango work of art and jessie ...
4,janice created an easy and agreeable work of art,Janice didn't create an easy work of art,Janice created an easy work of art,Janice created an agreeable work of art,1,Janice created an easy and agreeable work of a...,brenda created an easy work of art,jessie created an agreeable work of art,brenda created an easy work of art and jessie ...


In [46]:
df2_test_plus = df2_test[["sentence1_p", "sentence2", "label"]]

df2_test_plus.sentence2 = df2_test_plus.sentence2.map(normalizeString)

/home/felsal/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [47]:
df2_test_plus.rename(columns={"sentence1_p": "sentence1"}, inplace=True)

/home/felsal/.local/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [48]:
df2_test_plus.head()

,sentence1,sentence2,label
0,gregory created an ashy work of art and marion...,alexander didn t create an ashy work of art,0
1,brenda created a calm work of art and ted crea...,patricia didn t create a calm work of art,1
2,brenda created a beautiful work of art and jes...,juana didn t create a beautiful work of art,0
3,brenda created a mango work of art and jessie ...,herbert didn t create a mysterious work of art,1
4,brenda created an easy work of art and jessie ...,janice didn t create an easy work of art,1


In [49]:
df2_test_plus.to_csv("data/boolean5_plus_test.csv", index=False)